In [1]:
import numpy as np
import urllib2
import re
import os
import sys

from bs4 import BeautifulSoup

In [2]:
ROOT_URL = 'http://www.english-subtitles.pro/'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.87 Safari/537.36'
headers = { 'User-Agent' : user_agent }

In [8]:
def download(pageNum):
    page_url = ROOT_URL + '/movies/by-letters/page/' + str(pageNum)
    request = urllib2.Request(page_url, headers=headers)
    response = urllib2.urlopen(request)
    
    soup = BeautifulSoup(response.read(), 'lxml')
    
    movie_tags = [div_tag.find('a') for div_tag in soup.find_all("div", class_="entry-title")]
    movie_urls_names = [(tag['href'], tag.get_text()) for tag in movie_tags]
    
    success = 0
    for url, name in movie_urls_names:
        request = urllib2.Request(ROOT_URL + url, headers=headers)
        response = urllib2.urlopen(request)
        
        soup = BeautifulSoup(response.read(), 'lxml')
        
        aTags = soup.find_all(title='Free download')
        if(len(aTags) == 0): continue
        
        downloadTimes = [aTags[0].parent.parent.findChildren()[-2].get_text() for tag in aTags]
        maxIdx = np.argmax(downloadTimes)
        
        extension = os.path.splitext(aTags[maxIdx].get_text())[1]
        
        request = urllib2.Request(ROOT_URL + aTags[maxIdx]['href'], headers=headers)
        response = urllib2.urlopen(request)
        
        try:
            with open('english_subtitles/' + name + extension, 'w') as f:
                f.write(response.read())
            success += 1
        except:
            e = sys.exc_info()[0]
            print( "<p>Error: %s</p>, %s" % (e, name) )
    
    print ("Page {} finished with {} movies.".format(pageNum, success))

In [9]:
download(1) # Make it a loop to go over the 966 pages
# TODO: add proxy to avoid ip block

Page 1 finished with 47 movies.
